In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [2]:
url = 'https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases.html'

In [3]:
r = requests.get(url)
soup = BeautifulSoup(r.text)
latest_date = soup.td.get_text()

In [4]:
file_list = []
for tag in soup.find_all(href=re.compile("tar")):
    file_name = tag.get_text().replace('.tar.gz', '').split('_')[1]
    file_list.append(file_name)

In [ ]:
dataset_statistics = {}
for i in file_list:
    df = pd.read_csv('https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/' + i + '/metadata.csv', parse_dates=['publish_time'])
    df['author_list'] = df['authors'].astype(str).apply(lambda x: x.split(';'))
    df['author_count'] = df['author_list'].str.len()
    dataset_statistics[i] = df['author_count'].describe().tolist()

In [15]:
describe = ['Article Count', 'Mean Author Count', 'Std Author Count', 'Min Author Count', '25% Percentile', '50% Percentile', '75% Percentile', 'Max Author Count']
data = pd.DataFrame.from_dict(dataset_statistics, orient='index')
data.columns = describe

In [17]:
data.to_csv('~/github-actions-test/data/CORD-19-historical-releases.csv')